# Amazon Textract - a fully-managed AI Service for analyzing documents
#### A workshop on extracting text from documents

## Contents 
1. [Detect text from local image](#item1)
1. [Detect text from S3 object](#item2)
1. [Reading order](#item3)
1. [Natural Language Processing](#item4)
1. [Amazon Translate](#item5)
1. [Forms: Key-Value Pairs](#item6)
1. [Control and Compliance: Redaction](#item7)
1. [Table Validation](#item8)
1. [PDF Processing](#item9)

Install TRP - the Textract Results Parser  
Requires python 3.6 or later  
This python library makes it easier to navigate the JSON results from Textract analysis.
https://pypi.org/project/textract-trp/

In [ ]:
!pip install textract-trp

In [ ]:
import os
import boto3
import time
from IPython.display import Image, display, IFrame
from trp import Document
from PIL import Image as PImage, ImageDraw
from sagemaker import get_execution_role

In [ ]:
# Curent AWS Region. Use this to choose corresponding S3 bucket with sample content

mySession = boto3.session.Session()
awsRegion = mySession.region_name

print('Region:', awsRegion)
print('SageMaker Execution Role:', get_execution_role())

## Pause Here
#### and add the policy *AmazonTextractFullAccess* to the SageMaker Execution Role (shown above) used by this SageMaker Studio instance <hr>

#### S3 bucket that contains sample documents
We are providing sample documents in this bucket so you do not have to manually download/upload some of the test documents

In [ ]:
s3BucketName = "aws-workshops-" + awsRegion

In [ ]:
# Amazon S3 client
s3 = boto3.client('s3')

# Amazon Textract client
textract = boto3.client('textract')

# 1. Detect text from local image <a name="item1"></a>

https://docs.aws.amazon.com/textract/latest/dg/API_DetectDocumentText.html

In [ ]:
# Document
documentName = "./data/simple-document-image.jpg"
display(Image(filename=documentName))

In [ ]:
# Read document content
with open(documentName, 'rb') as document:
    imageBytes = bytearray(document.read())

# Call Amazon Textract
response = textract.detect_document_text(Document={'Bytes': imageBytes})

# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print (item["Text"])

# 2. Detect text from S3 object <a name="item2"></a>

https://docs.aws.amazon.com/textract/latest/dg/API_DetectDocumentText.html

In [ ]:
# Document
documentName = "textract-samples/simple-document-image.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })


# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print (item["Text"])

## Lines and Words of Text - JSON Structure

https://docs.aws.amazon.com/textract/latest/dg/API_BoundingBox.html  
https://docs.aws.amazon.com/textract/latest/dg/text-location.html  
https://docs.aws.amazon.com/textract/latest/dg/how-it-works-lines-words.html  

In [ ]:
# Document
documentName = "./data/OneLine.png"
display(Image(filename=documentName))

In [ ]:
# Read document content
with open(documentName, 'rb') as document:
    imageBytes = bytearray(document.read())

# Call Amazon Textract
response = textract.detect_document_text(Document={'Bytes': imageBytes})

# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print (item["Text"])

In [ ]:
print("JSON Response\n===================")
display(response)

# 3. Reading order <a name="item3"></a>

In [ ]:
# Document
documentName = "textract-samples/two-column-image.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

In [ ]:
# Detect columns and print lines
columns = []
lines = []
for item in response["Blocks"]:
      if item["BlockType"] == "LINE":
        column_found=False
        for index, column in enumerate(columns):
            bbox_left = item["Geometry"]["BoundingBox"]["Left"]
            bbox_right = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]
            bbox_centre = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]/2
            column_centre = column['left'] + column['right']/2

            if (bbox_centre > column['left'] and bbox_centre < column['right']) or (column_centre > bbox_left and column_centre < bbox_right):
                #Bbox appears inside the column
                lines.append([index, item["Text"]])
                column_found=True
                break
        if not column_found:
            columns.append({'left':item["Geometry"]["BoundingBox"]["Left"], 'right':item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]})
            lines.append([len(columns)-1, item["Text"]])

lines.sort(key=lambda x: x[0])
for line in lines:
    print (line[1])

# 4. Natural Language Processing <a name="item4"></a>

In [ ]:
# Document
documentName = "textract-samples/simple-document-image.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

#print(response)

# Print text
print("\nText\n========")
text = ""
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print ('\033[94m' +  item["Text"] + '\033[0m')
        text = text + " " + item["Text"]

# Amazon Comprehend client
comprehend = boto3.client('comprehend')

# Detect sentiment
sentiment =  comprehend.detect_sentiment(LanguageCode="en", Text=text)
print ("\nSentiment\n========\n{}".format(sentiment.get('Sentiment')))

# Detect entities
entities =  comprehend.detect_entities(LanguageCode="en", Text=text)
print("\nEntities\n========")
for entity in entities["Entities"]:
    print ("{}\t=>\t{}".format(entity["Type"], entity["Text"]))

# 5. Amazon Translate <a name="item5"></a>

In [ ]:
# Document
documentName = "textract-samples/simple-document-image.jpg"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

#### You will see an error when you run this next cell - this is part of the workshop
#### How can this error be fixed?

In [ ]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

# Amazon Translate client
translate = boto3.client('translate')

translation = []

print ('Original Language:')
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        print (item["Text"])
        result = translate.translate_text(Text=item["Text"], SourceLanguageCode="en", TargetLanguageCode="es")
        translation.append(result.get('TranslatedText'))
print ('\n')

print ('Spanish:')

for line in translation:
    print(line)


# 6. Forms: Key-Value Pairs <a name="item6"></a>

https://docs.aws.amazon.com/textract/latest/dg/API_AnalyzeDocument.html

In [ ]:
# Document
documentName = "textract-samples/employmentapp.png"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["FORMS"])

doc = Document(response)

for page in doc.pages:
    # Print fields
    print("Fields:")
    for field in page.form.fields:
        print("Key: {}\nValue: {}".format(field.key, field.value))

    # Get field by key
    print("\nGet Field by Key:")
    key = "Phone Number:"
    field = page.form.getFieldByKey(key)
    if(field):
        print("Key: {}\nValue: {}".format(field.key, field.value))

    # Search fields by key
    print("\nSearch Fields:")
    key = "address"
    fields = page.form.searchFieldsByKey(key)
    for field in fields:
        print("Key: {}\nValue: {}".format(field.key, field.value))

#### Form Data (Key-Value Pairs) JSON Structure

https://docs.aws.amazon.com/textract/latest/dg/how-it-works-kvp.html  
https://docs.aws.amazon.com/textract/latest/dg/how-it-works-selectables.html


In [ ]:
# Document
documentName = "./data/OneKeyValue.png"
display(Image(filename=documentName))

In [ ]:
# Read document content
with open(documentName, 'rb') as document:
    imageBytes = bytearray(document.read())

# Call Amazon Textract
response = textract.analyze_document(Document={'Bytes': imageBytes}, FeatureTypes=["FORMS"])

doc = Document(response)

for page in doc.pages:
    # Print fields
    print("Fields:")
    for field in page.form.fields:
        print("Key: {}\nValue: {}".format(field.key, field.value))

In [ ]:
print("JSON Response\n===================")
print(response)

# 7. Control and Compliance: Redaction <a name="item7"></a>

https://github.com/aws-samples/amazon-textract-response-parser  
https://github.com/aws-samples/amazon-textract-response-parser/blob/master/src-python/trp.py  

In [ ]:
# Document
documentName = "./data/employmentapp.png"
display(Image(filename=documentName))

In [ ]:
# Call Amazon Textract
with open(documentName, 'rb') as document:
    imageBytes = bytearray(document.read())

# Call Amazon Textract
response = textract.analyze_document(Document={'Bytes': imageBytes}, FeatureTypes=["FORMS"])

#print(response)

doc = Document(response)

# Redact document
img = PImage.open(documentName)

width, height = img.size

if(doc.pages):
    page = doc.pages[0]
    for field in page.form.fields:
        if(field.key and field.value and "address" in field.key.text.lower()):
            print("Redacting => Key: {}, Value: {}".format(field.key.text, field.value.text))
            
            x1 = field.value.geometry.boundingBox.left*width
            y1 = field.value.geometry.boundingBox.top*height-2
            x2 = x1 + (field.value.geometry.boundingBox.width*width)+5
            y2 = y1 + (field.value.geometry.boundingBox.height*height)+2

            draw = ImageDraw.Draw(img)
            draw.rectangle([x1, y1, x2, y2], fill="Black")

outputDocumentName = "./data/redacted-{}".format(os.path.basename(documentName))
img.save(outputDocumentName)
display(Image(filename=outputDocumentName))

# 9. Tables

In [ ]:
# Document
documentName = "textract-samples/employmentapp.png"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["TABLES"])

#print(response)

doc = Document(response)

for page in doc.pages:
     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))

#### Table JSON Structure
How it works

https://docs.aws.amazon.com/textract/latest/dg/how-it-works-tables.html

In [ ]:
print(response)

# 8. Table Data Validation <a name="item8"></a>

In [ ]:
# Document
documentName = "textract-samples/expense.png"
display(Image(url=s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName})))

In [ ]:
# Call Amazon Textract
response = textract.analyze_document(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    },
    FeatureTypes=["TABLES"])

In [ ]:
# use json.dumps(response, indent=4) to print out a python dictionary in a nice readable format
print(response)

In [ ]:
doc = Document(response)

def isFloat(input):
  try:
    float(input)
  except ValueError:
    return False
  return True

warning = ""
for page in doc.pages:
     # Print tables
    for table in page.tables:
        for r, row in enumerate(table.rows):
            itemName  = ""
            for c, cell in enumerate(row.cells):
                print("Table[{}][{}] = {}".format(r, c, cell.text))
                if(c == 0):
                    itemName = cell.text
                elif(c == 4 and isFloat(cell.text) and itemName != 'Total'):
                    value = float(cell.text)
                    if(value == 1500):
                        print(cell.text)
                        warning = "{} is greater than $1000.".format(itemName)
if(warning):
    print("\nReview needed:\n====================\n" + warning)

# 9. PDF Processing <a name="item9"></a>

https://docs.aws.amazon.com/textract/latest/dg/API_StartDocumentTextDetection.html  
https://docs.aws.amazon.com/textract/latest/dg/API_GetDocumentTextDetection.html  
https://docs.aws.amazon.com/textract/latest/dg/API_StartDocumentAnalysis.html  
https://docs.aws.amazon.com/textract/latest/dg/API_GetDocumentAnalysis.html  

In [ ]:
def startJob(s3BucketName, objectName):
    response = None
    response = textract.start_document_text_detection(
    DocumentLocation={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': objectName
        }
    })

    return response["JobId"]

def isJobComplete(jobId):
    response = textract.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))

    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = textract.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))

    return status

def getJobResults(jobId):

    pages = []
    response = textract.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']

    while(nextToken):
        response = textract.get_document_text_detection(JobId=jobId, NextToken=nextToken)

        pages.append(response)
        print("Resultset page recieved: {}".format(len(pages)))
        nextToken = None
        if('NextToken' in response):
            nextToken = response['NextToken']

    return pages

In [ ]:
# Document
documentName = "textract-samples/Amazon-Textract-Pdf.pdf"
IFrame(s3.generate_presigned_url('get_object', Params={'Bucket': s3BucketName, 'Key': documentName}), 900, 400)

In [ ]:
jobId = startJob(s3BucketName, documentName)
print("Started job with id: {}".format(jobId))
if(isJobComplete(jobId)):
    response = getJobResults(jobId)

# Print detected text
print('')
for resultPage in response:
    for item in resultPage["Blocks"]:
        if item["BlockType"] == "LINE":
            print (item["Text"])

## GitHub repo with samples in other languages

https://github.com/aws-samples/amazon-textract-code-samples  